# Species Distribution Modeling with Occurrence Data

<img src="https://oceanhackweek.org/assets/img/ohw-logo.png" width="200" align="right">

This notebook follows the [Ocean HackWeek tutorial](https://oceanhackweek.org/tutorials_marine_sdm/tutorial/Steps_occurences.html) on Species Distribution Modeling (SDM) using occurrence data.

**Learning Objectives:**
1. Obtain and clean species occurrence data
2. Acquire and process environmental raster data
3. Prepare data for SDM modeling
4. Build and evaluate a species distribution model
5. Visualize habitat suitability predictions

**Species:** Blue Shark (*Prionace glauca*)  
**Region:** North Atlantic Ocean  
**Environmental Variables:** Bathymetry and Sea Surface Temperature

## 1. Setup Environment

In [5]:
# Install required packages (uncomment if needed)
#!pip install pandas geopandas rasterio matplotlib scikit-learn contextily numpy seaborn pyimpute mapclassify
#pip install robis

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.mask import mask
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
import contextily as ctx
from pyimpute import load_training_vector, load_targets, impute
import tempfile
from shapely.geometry import Point, box

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('seaborn-whitegrid')
sns.set(style="whitegrid")

## 2. Obtain and Prepare Occurrence Data

We'll use occurrence records for Blue Shark from GBIF.

In [ ]:
# Download occurrence data
occ_url = "https://raw.githubusercontent.com/OceanHackWeek/ohw-tutorials/main/marine_sdm/data/species_data.csv"
occ_path = "blue_shark_occurrences.csv"

# Download if not already present
if not os.path.exists(occ_path):
    !wget -O {occ_path} {occ_url}

# Load data
occurrences = pd.read_csv(occ_path)
print(f"Loaded {len(occurrences)} occurrence records")

# Show data structure
occurrences.head()

In [ ]:
# Basic cleaning
print("Cleaning occurrence data...")
original_count = len(occurrences)

# 1. Remove records without coordinates
occurrences = occurrences.dropna(subset=['decimalLongitude', 'decimalLatitude'])

# 2. Remove records with 0,0 coordinates
occurrences = occurrences[(occurrences.decimalLongitude != 0) | (occurrences.decimalLatitude != 0)]

# 3. Remove duplicate coordinates
occurrences = occurrences.drop_duplicates(subset=['decimalLongitude', 'decimalLatitude'])

# 4. Filter to study area (North Atlantic)
study_area = box(-100, 0, 0, 70)  # Approximate North Atlantic bounds
geometry = [Point(xy) for xy in zip(occurrences.decimalLongitude, occurrences.decimalLatitude)]
gdf_occurrences = gpd.GeoDataFrame(occurrences, geometry=geometry, crs="EPSG:4326")
gdf_occurrences = gdf_occurrences[gdf_occurrences.within(study_area)]

print(f"Retained {len(gdf_occurrences)} of {original_count} records after cleaning")

# Visualize occurrences
fig, ax = plt.subplots(figsize=(12, 8))
gdf_occurrences.plot(ax=ax, color='red', markersize=8, alpha=0.7, label='Occurrence')
ctx.add_basemap(ax, crs=gdf_occurrences.crs.to_string(), source=ctx.providers.Esri.OceanBasemap)
ax.set_title("Blue Shark Occurrence Records in North Atlantic")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.legend()
plt.show()

## 3. Acquire Environmental Data

We'll use bathymetry and sea surface temperature (SST) rasters.

In [ ]:
def download_raster(url, filename):
    """Download a raster file if not already present"""
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        !wget -O {filename} {url}
    else:
        print(f"{filename} already exists")
    return filename

In [ ]:
# Download bathymetry data
bathy_url = "https://github.com/OceanHackWeek/ohw-tutorials/raw/main/marine_sdm/data/bathymetry.tif"
bathy_path = download_raster(bathy_url, "bathymetry.tif")

# Download sea surface temperature data
sst_url = "https://github.com/OceanHackWeek/ohw-tutorials/raw/main/marine_sdm/data/sst.tif"
sst_path = download_raster(sst_url, "sst.tif")

In [ ]:
# Visualize environmental rasters
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bathymetry
with rasterio.open(bathy_path) as src:
    bathy = src.read(1)
    show(src, ax=ax1, title='Bathymetry (m)', cmap='Blues_r')
    ax1.set_xlabel("Longitude")
    ax1.set_ylabel("Latitude")

# Sea Surface Temperature
with rasterio.open(sst_path) as src:
    sst = src.read(1)
    show(src, ax=ax2, title='Sea Surface Temperature (°C)', cmap='plasma')
    ax2.set_xlabel("Longitude")
    ax2.set_ylabel("Latitude")

plt.tight_layout()
plt.show()

## 4. Data Processing

We'll extract environmental values at occurrence points and generate background points.

In [ ]:
def extract_raster_values(gdf, raster_path, band=1, nodata=np.nan):
    """Extract raster values at point locations"""
    values = []
    with rasterio.open(raster_path) as src:
        for point in gdf.geometry:
            x, y = point.x, point.y
            if src.bounds.left <= x <= src.bounds.right and src.bounds.bottom <= y <= src.bounds.top:
                row, col = src.index(x, y)
                window = ((row, row+1), (col, col+1))
                data = src.read(band, window=window)
                if data.size > 0 and data[0, 0] != src.nodata:
                    values.append(data[0, 0])
                else:
                    values.append(nodata)
            else:
                values.append(nodata)
    return values

In [ ]:
print("Extracting environmental values at occurrence points...")
gdf_occurrences['bathy'] = extract_raster_values(gdf_occurrences, bathy_path)
gdf_occurrences['sst'] = extract_raster_values(gdf_occurrences, sst_path)

# Remove points with missing environmental data
original_occ_count = len(gdf_occurrences)
gdf_occurrences = gdf_occurrences.dropna(subset=['bathy', 'sst'])
print(f"Retained {len(gdf_occurrences)} of {original_occ_count} occurrence points with valid environmental data")

In [ ]:
print("Generating background points...")
minx, miny, maxx, maxy = gdf_occurrences.total_bounds
n_background = len(gdf_occurrences) * 10  # 10:1 background-to-presence ratio

# Create random points within bounds
background_points = gpd.GeoDataFrame(
    geometry=[Point(x, y) for x, y in zip(
        np.random.uniform(minx, maxx, n_background),
        np.random.uniform(miny, maxy, n_background)
    )],
    crs="EPSG:4326"
)

# Extract environmental data for background points
background_points['bathy'] = extract_raster_values(background_points, bathy_path)
background_points['sst'] = extract_raster_values(background_points, sst_path)
background_points['presence'] = 0

# Clean background points
original_bg_count = len(background_points)
background_points = background_points.dropna(subset=['bathy', 'sst'])
print(f"Retained {len(background_points)} of {original_bg_count} background points with valid environmental data")

In [ ]:
# Combine datasets
gdf_occurrences['presence'] = 1
combined = pd.concat([
    gdf_occurrences[['presence', 'bathy', 'sst', 'geometry']],
    background_points[['presence', 'bathy', 'sst', 'geometry']]
], ignore_index=True)

# Visualize environmental space
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x='bathy', 
    y='sst', 
    hue='presence', 
    data=combined, 
    palette={0: 'blue', 1: 'red'}, 
    alpha=0.3
)
plt.title("Environmental Space: Presence vs Background")
plt.xlabel("Bathymetry (m)")
plt.ylabel("Sea Surface Temperature (°C)")
plt.legend(title="Presence", labels=['Background', 'Blue Shark'])
plt.show()

## 5. Model Training

We'll train a Random Forest classifier to predict species occurrence.

In [ ]:
# Prepare data for modeling
X = combined[['bathy', 'sst']]
y = combined['presence']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)} ({len(y_train[y_train==1])} presence)")
print(f"Testing samples: {len(X_test)} ({len(y_test[y_test==1])} presence)")

In [ ]:
# Train Random Forest model
print("Training Random Forest model...")
model = RandomForestClassifier(
    n_estimators=200, 
    max_depth=10, 
    random_state=42, 
    class_weight='balanced',
    n_jobs=-1
)
model.fit(X_train, y_train)

# Evaluate model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f"Training accuracy: {train_score:.4f}")
print(f"Testing accuracy: {test_score:.4f}")

In [ ]:
# Calculate AUC
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"Model AUC: {auc:.4f}")

# Confusion matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Background', 'Presence'],
            yticklabels=['Background', 'Presence'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Feature importance
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(8, 4))
sns.barplot(x='Importance', y='Feature', data=feature_importances, palette='viridis')
plt.title('Feature Importance')
plt.show()

## 6. Habitat Suitability Mapping

Predict habitat suitability across the entire study area.

In [ ]:
# Create a function for raster prediction
def predict_raster(model, raster_paths, output_path, nodata=-9999):
    """Predict habitat suitability across raster layers"""
    # Load target rasters
    raster_stack = []
    with rasterio.open(raster_paths[0]) as src:
        profile = src.profile
        transform = src.transform
        height, width = src.shape
        raster_stack.append(src.read(1))
    
    for path in raster_paths[1:]:
        with rasterio.open(path) as src:
            raster_stack.append(src.read(1))
    
    # Stack rasters into a 3D array
    stacked = np.dstack(raster_stack)
    original_shape = stacked.shape[:2]
    stacked = stacked.reshape(-1, stacked.shape[2])
    
    # Predict
    print("Predicting habitat suitability...")
    suitability = model.predict_proba(stacked)[:, 1]
    suitability = suitability.reshape(original_shape)
    
    # Set nodata values
    mask = np.isnan(stacked[:, 0]) | (stacked[:, 0] == nodata)
    suitability.flat[mask] = nodata
    
    # Save output
    profile.update(
        dtype=rasterio.float32,
        count=1,
        nodata=nodata,
        compress='lzw'
    )
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(suitability.astype(rasterio.float32), 1)
    
    return suitability

In [ ]:
# Predict habitat suitability
suitability_path = "habitat_suitability.tif"
suitability = predict_raster(model, [bathy_path, sst_path], suitability_path)

In [ ]:
# Visualize habitat suitability
with rasterio.open(suitability_path) as src:
    suitability = src.read(1)
    profile = src.profile
    
    # Create a custom colormap
    cmap = plt.cm.viridis
    cmap.set_bad('lightgrey', 1.0)  # Color for nodata
    
    plt.figure(figsize=(12, 10))
    im = plt.imshow(suitability, cmap=cmap, 
                   vmin=0, vmax=1, 
                   extent=[src.bounds.left, src.bounds.right, 
                           src.bounds.bottom, src.bounds.top])
    
    # Add occurrence points
    gdf_occurrences.plot(ax=plt.gca(), color='red', markersize=5, alpha=0.7)
    
    # Add colorbar
    cbar = plt.colorbar(im, fraction=0.025, pad=0.04)
    cbar.set_label('Habitat Suitability Probability')
    
    plt.title("Blue Shark Habitat Suitability in North Atlantic")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(False)
    plt.show()

## 7. Model Interpretation

Analyze the response curves to understand how environmental variables affect habitat suitability.

In [ ]:
# Create response curves
def plot_response_curve(model, feature_name, feature_range, ax, other_feature_value=0):
    """Plot response curve for a given feature"""
    # Create synthetic data
    synthetic = np.zeros((len(feature_range), 2))
    synthetic[:, 0] = other_feature_value  # Hold other feature constant
    synthetic[:, 1] = feature_range
    
    # Swap columns based on feature position
    if feature_name == 'bathy':
        synthetic = synthetic[:, [0, 1]]
    else:
        synthetic = synthetic[:, [1, 0]]
    
    # Predict
    predictions = model.predict_proba(synthetic)[:, 1]
    
    # Plot
    ax.plot(feature_range, predictions, linewidth=2)
    ax.set_title(f"Response Curve: {feature_name}")
    ax.set_xlabel(feature_name)
    ax.set_ylabel("Probability of Occurrence")
    ax.grid(True)

In [ ]:
# Generate response curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bathymetry response curve
bathy_range = np.linspace(
    combined['bathy'].min(), 
    combined['bathy'].max(), 
    100
)
plot_response_curve(model, 'bathy', bathy_range, ax1, 
                    other_feature_value=combined['sst'].median())

# SST response curve
sst_range = np.linspace(
    combined['sst'].min(), 
    combined['sst'].max(), 
    100
)
plot_response_curve(model, 'sst', sst_range, ax2, 
                    other_feature_value=combined['bathy'].median())

plt.tight_layout()
plt.show()

## Conclusion

This notebook demonstrated a complete workflow for marine species distribution modeling:
1. Obtained and cleaned occurrence data for Blue Shark
2. Acquired and processed environmental raster data (bathymetry and SST)
3. Prepared data for modeling by extracting environmental values and generating background points
4. Trained and evaluated a Random Forest model
5. Predicted and visualized habitat suitability across the study area
6. Interpreted model results using response curves

The model achieved an AUC of {auc:.3f}, indicating good predictive performance. The habitat suitability map shows that Blue Sharks prefer deeper, warmer waters in the North Atlantic.